In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

def load_data(file_path):
    # Load the dataset
    data = pd.read_csv(file_path, header=None)
    X = data.iloc[:, :-1].values  # Features (all columns except the last one)
    y = data.iloc[:, -1].values   # Labels (last column)
    y = y.reshape(-1, 1)  # Reshape labels for compatibility
    return X, y

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

def initialize_weights(input_size, hidden1_size, hidden2_size, output_size):
    # Initialize weights randomly
    weights = {
        'W1': np.random.rand(input_size, hidden1_size),
        'W2': np.random.rand(hidden1_size, hidden2_size),
        'W3': np.random.rand(hidden2_size, output_size)
    }
    return weights

def forward_propagation(X, weights):
    # Input to hidden layer 1
    z1 = np.dot(X, weights['W1'])
    a1 = sigmoid(z1)

    # Hidden layer 1 to hidden layer 2
    z2 = np.dot(a1, weights['W2'])
    a2 = sigmoid(z2)

    # Hidden layer 2 to output layer
    z3 = np.dot(a2, weights['W3'])
    a3 = sigmoid(z3)

    return a1, a2, a3

def backward_propagation(X, y, a1, a2, a3, weights, learning_rate):
    # Compute gradients
    error = a3 - y
    delta3 = error * sigmoid_derivative(a3)
    dW3 = np.outer(a2, delta3)  # Use outer product to match dimensions

    delta2 = np.dot(delta3, weights['W3'].T) * sigmoid_derivative(a2)
    dW2 = np.outer(a1, delta2)

    delta1 = np.dot(delta2, weights['W2'].T) * sigmoid_derivative(a1)
    dW1 = np.outer(X, delta1)

    # Update weights
    weights['W1'] -= learning_rate * dW1
    weights['W2'] -= learning_rate * dW2
    weights['W3'] -= learning_rate * dW3

    return weights

def train_neural_network_with_early_stopping(X_train, y_train, input_size, hidden1_size, hidden2_size, output_size, epochs, learning_rate, patience=10):
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    weights = initialize_weights(input_size, hidden1_size, hidden2_size, output_size)
    
    best_val_accuracy = 0
    best_weights = weights
    count = 0  # Counter for early stopping
    
    for epoch in range(epochs):
        for i in range(len(X_train)):
            # Forward propagation
            a1, a2, a3 = forward_propagation(X_train[i], weights)

            # Backward propagation
            weights = backward_propagation(X_train[i], y_train[i], a1, a2, a3, weights, learning_rate)

        # Evaluate on the validation set
        val_predictions = predict(X_val, weights)
        val_accuracy = np.mean((val_predictions > 0.5) == y_val)

        # Check for improvement
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_weights = weights
            count = 0
        else:
            count += 1

        # Early stopping
        if count >= patience:
            break

    return best_weights

def predict(X, weights):
    _, _, predictions = forward_propagation(X, weights)
    return predictions

# Load training data

train_file_path = "C:\\Users\\santhosh\\Downloads\\project\\santhosh\\Neural-Networks\\dataset\\bank-note\\train.csv"
test_file_path = "C:\\Users\\santhosh\\Downloads\\project\\santhosh\\Neural-Networks\\dataset\\bank-note\\test.csv"


X_train, y_train = load_data(train_file_path)
X_test, y_test = load_data(test_file_path)

# Assuming X_train, y_train, X_test, and y_test are the training and test data and labels
input_size = X_train.shape[1]
hidden1_size = 4  # Adjust as needed
hidden2_size = 3  
output_size = 1
learning_rate = 0.01
epochs = 1000
patience = 20  

# Train the neural network with early stopping
best_weights = train_neural_network_with_early_stopping(X_train, y_train, input_size, hidden1_size, hidden2_size, output_size, epochs, learning_rate, patience)

# Test the neural network on the test data
predictions = predict(X_test, best_weights)

# Evaluate the performance
accuracy = np.mean((predictions > 0.5) == y_test)
print(f"Accuracy on test data: {accuracy * 100:.2f}%")


Accuracy on test data: 55.80%
